In [101]:
import psycopg
import json
from shapely.geometry import shape

In [24]:
def run_query(query, fetch_results = False):
    with psycopg.connect(dbname='taxi_availability', user='myuser', password='mypassword', host='localhost', port='5432') as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            if fetch_results:
                try:
                    results = cur.fetchall()
                    return results
                except:
                    print("Query does not produce any results")

In [93]:
query = """
    CREATE TABLE IF NOT EXISTS taxi_availability (
    timestamp timestamptz PRIMARY KEY,
    geom GEOMETRY(MultiPoint, 4326),
    taxi_count integer
);  
        """

In [94]:
run_query(query, True)

Query does not produce any results


In [95]:
with open("data/raw/taxi_availability_2024_12_31.json", "r") as file:
    taxi_availability_list = json.load(file)

In [102]:
conn = psycopg.connect(dbname='taxi_availability', user='myuser', password='mypassword', host='localhost', port='5432')
cur = conn.cursor()
insert_query = "INSERT INTO taxi_availability (timestamp, geom, taxi_count) VALUES (%s, ST_GeomFromText(%s, 4326), %s) \
                ON CONFLICT (timestamp) DO NOTHING;"
for taxi_availability in taxi_availability_list:
    geometry = shape(taxi_availability["features"][0]["geometry"])
    timestamp = taxi_availability["features"][0]["properties"]["timestamp"]
    taxi_count = taxi_availability["features"][0]["properties"]["taxi_count"]
    cur.execute(insert_query, (timestamp, geometry.wkt, taxi_count))
conn.commit()
cur.close()
conn.close()

In [105]:
results = run_query("select ST_AsText(geom) from taxi_availability", True)

In [106]:
results[0]

('MULTIPOINT((103.62857 1.30203),(103.64316 1.33408),(103.6685 1.31986),(103.67955 1.31969),(103.68565 1.34116),(103.68577 1.34729),(103.68882 1.34197),(103.68896 1.3448),(103.68965 1.34415),(103.69059 1.34),(103.6917 1.31534),(103.69332 1.3403),(103.6944 1.34),(103.69508 1.34327),(103.69523 1.34357),(103.69574 1.31973),(103.69627 1.31725),(103.69645 1.33185),(103.6966 1.32909),(103.69775 1.35),(103.69799 1.34835),(103.69801 1.35),(103.6980947724 1.317124899),(103.69848 1.35613),(103.69857 1.32),(103.69866 1.33),(103.69885 1.32085),(103.6989 1.33714),(103.69913 1.35),(103.69991 1.3483),(103.69994 1.34),(103.70048 1.35),(103.70083 1.34058),(103.70131 1.34),(103.70252 1.34),(103.70332 1.34454),(103.70522 1.34),(103.70573 1.3356),(103.70694 1.34),(103.70733 1.33942),(103.70742 1.33957),(103.70781 1.34384),(103.70791 1.34467),(103.70837 1.36601),(103.70854 1.35),(103.70856 1.32872),(103.70866 1.37058),(103.70961 1.35),(103.70975 1.34502),(103.70975 1.35),(103.70992 1.32552),(103.71123 1.35